#### To begin, run the cell below. It contains all of the necessary set-up code. 

In [206]:
import mysql.connector
import sqlalchemy as db
from urllib.parse import urlparse
from datetime import datetime
import pandas as pd
import numpy as np
import sys

#database path
f = open("db_path.txt", "r")
path = f.read()
f.close()

#create sqlalchemy engine for pd.read_sql
engine = db.create_engine(path)

#parse database URL
url = urlparse(path)

host = url.hostname
port = url.port
user = url.username
password = url.password
database = url.path[1:]

#define function to connect to the database
db_connect = lambda: mysql.connector.connect (host = host,
                                              port = port,
                                              user = user,
                                              password = password,
                                              database = database)

def write_query(db_connect, query, values, many = False):
    
    with db_connect() as connection:
        with connection.cursor() as cursor:
            
            if many:
                cursor.executemany(query, values)
            else:
                cursor.execute(query, values)

            connection.commit()
                   

def check_lease_name (lease_name):
    assert type(lease_name) == str, 'The lease_name is not a string'
    assert len(lease_name) <= 50, 'The lease_name is too long'
    assert lease_name not in list(leases['lease_name']), 'This lease_name has already been used'

def check_dates (j):
    for i in j:
        assert type(i) == str, f"{i} is not a string"
        try:
            pd.to_datetime(i)
        except:
            raise AssertionError(f"{i} is not a valid date or is not in the valid date format.")
            
def check_numbers(j):
    for i in j:
        assert type(i) == int or type(i) == float, f"{i} is not an integer or float"
        assert i >=0, 'There should not be any negative inputs'

def check_bools(j):
    for i in j:
        assert type(i) == bool, f"{i} is not a boolean" 

def print_lease_info():
    print('\n')
    print('Lease Name:', lease_name)
    print('Lease Commencement:', commencement_date)
    print('Lease Termination:', termination_date)
    print('Upfront Payments:', upfront_payments)
    print('Termination Payments:', termination_payments)
    print('Discount Rate:', discount_rate)
    print('Useful Life of Asset:', useful_life_of_asset)
    print('Fair Value of Asset:', fair_value_of_asset)
    print('Economic Life Policy:', economic_life_policy)
    print('Fair Value Policy:', fair_value_policy)
    print('Ownership Transfer:', ownership_transfer)
    print('Purchase Option:', purchase_option)
    print('Specialized Asset:', specialized_asset)     

class payment_schedule:

    def __init__(self, lease_start, lease_end, start_payments, end_payments):
        
        self.lease_start = pd.to_datetime(lease_start)
        self.lease_end = pd.to_datetime(lease_end)
        self.start_payments = start_payments
        self.end_payments = end_payments

        self.schedule = pd.DataFrame([self.lease_start, self.lease_end], columns = ['payment_date'])
        self.schedule['payment_amount'] = [self.start_payments, self.end_payments]
    
    def add_payments (self, payment, start_month, day_of_month = 1, repeat = 1, end_of_month = False):
        
        assert type(payment) == int or type(payment) == float, 'Please enter a valid payment amount'
        assert type(start_month) == str, 'Please enter the start month as a string'
        try:
            pd.to_datetime(start_month)
        except:
            print('Please enter a valid date in YYYY-MM format for the start month')
        assert len(start_month) == 7, 'The date must be in YYYY-MM format for the start month'
        assert type(repeat) == int, 'Please enter an integer for the repeat argument'
        assert repeat > 0, 'The repeat must be at least 1'
        assert type(end_of_month) == bool, 'Please enter True or False for the end_of_month argument'
        assert type(day_of_month) == int, 'Please enter an integer for the alt_day argument'
        assert day_of_month <=31 and day_of_month >=1, 'The alt_day argument does not appear to be a valid day of the month'
        
        #define start
        start = pd.to_datetime(start_month)

        #define payment frequency
        frequency = 'MS'
        
        if end_of_month:
            frequency = 'M'
            day_of_month = 1

            
        #create the date range
        date_range = pd.Series(pd.date_range(start, freq = frequency, periods = repeat) + pd.DateOffset(day_of_month-1))
        
        if date_range[0] >= pd.Timestamp(self.lease_start) and date_range[repeat-1] <= pd.Timestamp(self.lease_end):
            
            temp_df = pd.DataFrame(date_range, columns = ['payment_date'])
            temp_df['payment_amount'] = payment
            temp_df = pd.concat([self.schedule, temp_df])
            temp_df = pd.DataFrame(temp_df.groupby(temp_df['payment_date'])['payment_amount'].sum()).reset_index()
            temp_df = temp_df.sort_values(by = 'payment_date')
            temp_df['no_payment_flag'] = 1
            temp_df['no_payment_flag'] = np.where((temp_df['payment_date'] != self.lease_start) &
                                                   (temp_df['payment_date'] != self.lease_end) &
                                                   (temp_df['payment_amount'] <= 0), 0, temp_df['no_payment_flag'])
            
            temp_df = pd.DataFrame(temp_df.loc[temp_df['no_payment_flag']==1])
            temp_df = temp_df.drop(columns = ['no_payment_flag'])
            temp_df = temp_df.reset_index(drop = True)
            
            self.schedule = temp_df
            print('Payments have been added to the schedule.')

        else:
            return('You have entered payments outside of the lease term.')
    
    def reset_schedule (self):
        self.schedule = pd.DataFrame([self.lease_start, self.lease_end], columns = ['payment_date'])
        self.schedule['payment_amount'] = [self.start_payments, self.end_payments]
    

## Step 1 - Select a Company

### Select an Existing Company

1. Run the cell below to view existing companies.

In [211]:
companies = pd.read_sql_query ('select * from companies', engine.connect())
pd.set_option('display.max_rows', None)
display(companies)

,company_id,company_name
0,35,ABC Corporation
1,25,Quantum Leap Innovations
2,26,Verdant Horizons Corporation
3,27,Zenith Digital Solutions


2. Enter the desired company's name as a string and run the cell.

In [212]:
company_name = 'ABC Corporation'

3. Run the cell below. If you have entered a valid company, it will select the company.

In [213]:
try:
    company_id = companies.loc[companies['company_name']==company_name, 'company_id'].item()
    print(f"You have successfully selected {company_name}. Please proceed to Step 2 or select a different company.")
except:
    print('The company name selected does not appear in the table above. Please enter a valid company name.')

You have successfully selected ABC Corporation. Please proceed to Step 2 or select a different company.


### OR Create a New Company

1. Enter the name of the new company below:
> - The name must be 50 characters or less.
> - The name must not be a duplicate name from the table above.

In [208]:
new_company_name = 'ABC Corporation'

2. Run the cell below to ensure that the new company name is a valid entry.

In [209]:
assert type(new_company_name) == str, 'The name must be a string.'
assert new_company_name not in list(companies['company_name']), 'This name appears to be a duplicate.'
assert len (new_company_name) <= 50, 'This name is too long. It must be 50 characters or less.'

print(f"{new_company_name} appears to be a valid name. Please proceed.")

ABC Corporation appears to be a valid name. Please proceed.


3. If the name appears valid, run the cell below to create a new company. 

In [210]:
assert type(new_company_name) == str, 'The name must be a string.'
assert new_company_name not in list(companies['company_name']), 'This name appears to be a duplicate.'
assert len (new_company_name) <= 50, 'This name is too long. It must be 50 characters or less.'

try:
    query = 'INSERT INTO companies (company_name) VALUES (%s)'
    values = [new_company_name]
    write_query (db_connect, query, values)
    print('A new company has been successfully created.')
except mysql.connector.Error as e:
    print('an error occurred with the database query and no new company was created. See MySQL error below:')
    print(e)
        

A new company has been successfully created.


#### If a new company has been succesfully created, return to the start of "Select a Company" and select the desired company from the company table.

## Step 2 - Select a Lease

Begin this step after successfully selecting a company. In this step, you can add/delete leases or view/edit existing lease information (start/end date, discount rate, classification criteria, etc.). Lease payment schedules are evaluated separately further below.

### Select an Existing Lease

1. Run the cell below to view the company's existing leases.

> <br> <b>To add a lease, continue to "Create a New Lease" below.</b>

In [225]:
leases = pd.read_sql_query ('select * from leases WHERE company_id = %(identifier)s', engine.connect(), params = {'identifier':company_id})
pd.set_option('display.max_rows', None)
display(leases[['lease_id','lease_name']])

,lease_id,lease_name
0,33,Example Lease A


2. Enter the desired lease's id as an integer and run the cell.

In [226]:
lease_id = 33

3. Run the cell below. If a valid lease id was entered, it will select the lease.

In [227]:
try:
    lease = leases.loc[leases['lease_id']==lease_id, 'lease_name'].item()
    print(f"You have selected the following lease: {lease}")
    print('You may now view/edit this lease.')
except:
    print('The lease id entered does not appear in the table above. Please enter a valid lease id.')

You have selected the following lease: Example Lease A
You may now view/edit this lease.


#### View Lease Details

Run the cell below to view the details previously input for the lease. Boolean fields are presented as "1" for True and "0" for False.

In [ ]:
cols = leases.columns[1:]
leases.loc[leases['lease_name']==lease][cols]

#### Edit Lease Details

To edit any of the information above, provide the name of the column to update (col_name) as a string, and the new value (new_val) as the relevant data type (enter True or False for boolean fields). 

In [ ]:
col_name = 'ownership_transfer'
new_val = True

Run the following cell to update the lease information. 

In [ ]:
valid_names = ['lease_name', 'commencement_date', 'termination_date', 'upfront_payments', 'termination_payments',
               'discount_rate', 'useful_life_of_asset', 'fair_value_of_asset', 'economic_life_policy', 'fair_value_policy',
               'ownership_transfer', 'purchase_option', 'specialized_asset']

assert col_name in valid_names, 'This is not a valid column name, or you have selected a column that cannot be updated.'

if col_name == valid_names[0]:
    check_lease_name(new_val)
    
if col_name in valid_names[1:3]:
    check_dates([new_val])

if col_name in valid_names[3:10]:
    check_numbers([new_val])

if col_name in valid_names[10:]:
    check_bools([new_val])

query = f"UPDATE leases SET {col_name} = %s WHERE lease_name = %s;"
values = (new_val, lease)

try:
    write_query (db_connect, query, values)
    print('Lease information has been updated. Return to the top of Step 2 and re-select the lease to view changes.')
except mysql.connector.Error as e:
    print('an error occurred with the database query and no new company was created. See MySQL error below:')
    print(e)

### Create a New Lease

To create a new lease, enter all of the required information below, noting the specifications.
<br>
<br>
After creating a lease, a lease payment schedule will need to be created in <b> Step 3 </b> below before the lease is ready to be evaluated in Part II. 

1. Enter the lease name as a string. The name must be 50 characters or less, and must not be a duplicate name.

In [215]:
lease_name = 'Example Lease A'

2. Enter the lease commencement and termination dates. The dates must be strings in the YYYY-MM-DD format.

In [216]:
commencement_date = '2020-04-01'
termination_date = '2028-03-31'

3. Enter any payments made on or before lease initiation (prepayments, initial upfront payments, etc.). The amount should be an integer or float. If there are no upfront payments, enter 0.

In [217]:
upfront_payments = 0

4. Enter any payments made upon lease termination (such as the amount of a purchase option that is likely to be exercised). The amount should be an integer or a float. If there are no payments made upon lease termination, enter 0.

In [218]:
termination_payments = 0

5. Enter the discount rate applicable to the lease <b> as a decimal </b>. Using a % sign will not work.

In [219]:
discount_rate = 0.06

6. Enter the estimated useful life of the leased asset <b> in months </b> and an estimate of fair value of the leased asset. Both inputs should be an integer or a float. This data is required to assess the lease classification.

In [220]:
useful_life_of_asset = 10000
fair_value_of_asset = 1000000000

7. Each company must determine what they consider to constitute a "major part" of the economic life or fair value of the leased asset. A rule-of-thumb is 75% for the useful life election, and 90% for the fair value election. Enter these amounts as decimals below or, if the company has a different policy election, enter the correct amounts. The resulting data type should be float.

In [221]:
economic_life_policy = 0.75
fair_value_policy = 0.90

8. Enter boolean data (True or False) for each lease classification criteria listed below. 
> - If the lease terms include a transfer of ownership of the leased asset, enter True for 'ownership_transfer'. 
> - If the lease includes a purchase option that the lessee is reasonably likely to exercise, enter True for 'purchase_option'.
> - If the leased asset is of a highly specialized nature, or other circumstances exist such that the leased asset will have no value to the lessor following the termination of the lease, enter True for 'specialized_asset'.
> - Enter False for any criteria that do not apply.

In [222]:
ownership_transfer = False
purchase_option = False
specialized_asset = False

#### Run the cell below to check the inputs above. It will check that proper data types/formats have been used.

In [223]:
check_lease_name (lease_name)

j = [commencement_date, termination_date]
check_dates(j)

j = [upfront_payments, termination_payments, discount_rate, useful_life_of_asset, fair_value_of_asset,
     economic_life_policy, fair_value_policy]
check_numbers(j)

j = [ownership_transfer, purchase_option, specialized_asset]
check_bools(j)

print('Lease inputs appear appropriate:')
print('Lease inputs are as follows:')

print_lease_info()

Lease inputs appear appropriate:
Lease inputs are as follows:


Lease Name: Example Lease A
Lease Commencement: 2020-04-01
Lease Termination: 2028-03-31
Upfront Payments: 0
Termination Payments: 0
Discount Rate: 0.06
Useful Life of Asset: 10000
Fair Value of Asset: 1000000000
Economic Life Policy: 0.75
Fair Value Policy: 0.9
Ownership Transfer: False
Purchase Option: False
Specialized Asset: False


#### Double check that the cell above does not return an error and that the inputs are accurate, then run the cell below to create a new lease.

In [224]:
check_lease_name (lease_name)

j = [commencement_date, termination_date]
check_dates(j)

j = [upfront_payments, termination_payments, discount_rate, useful_life_of_asset, fair_value_of_asset,
     economic_life_policy, fair_value_policy]
check_numbers(j)

j = [ownership_transfer, purchase_option, specialized_asset]
check_bools(j)

query = '''
INSERT INTO leases(company_id,lease_name,commencement_date,termination_date,upfront_payments,termination_payments,discount_rate,
useful_life_of_asset,fair_value_of_asset,economic_life_policy,fair_value_policy,ownership_transfer,purchase_option,
specialized_asset) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);'''

values = [company_id,lease_name, commencement_date, termination_date, upfront_payments, 
          termination_payments, discount_rate, useful_life_of_asset, fair_value_of_asset,
          economic_life_policy, fair_value_policy, ownership_transfer, purchase_option,
          specialized_asset]
try:
    write_query (db_connect, query, values)
    print('A new lease has been successfully created. After selecting the lease(above), go to Step 3 to add a payment schedule for the lease')
except mysql.connector.Error as e:
    print('an error occurred with the database query and no new company was created. See MySQL error below:')
    print(e)

A new lease has been successfully created. After selecting the lease(above), go to Step 3 to add a payment schedule for the lease


## Step 3 - Create/Edit Payment Schedules

To begin this step, first select a lease above. In this step, you can create/edit lease payment schedules.

### View/Edit the Payment Schedule
1. After selecting a lease, run the cell below retreive the lease's payment schedule.

In [228]:
retreive = lambda column: leases.loc[leases['lease_name']==lease, column].item()
pmt_df = pd.read_sql_query ('select * from payments WHERE lease_id = %(identifier)s', engine.connect(), params = {'identifier':retreive('lease_id')})

if len(pmt_df) == 0:
    print('No payment information has been created for this lease. You need to add payment information below.')
else:
    print('The following payments schedule has been assigned to the lease:')
    display(pmt_df[['payment_date','payment_amount']])

No payment information has been created for this lease. You need to add payment information below.


If you need to add/edit payment information, proceed with the steps below. Note that there is currently no way to edit payment information without creating a new payment schedule from scratch and overwriting the old one (sorry). 

> <b>NOTE:</b> If you update relevant lease information, such as the start/end date of the lease or the upfront/termination payment amounts, then those updates will not automatically flow through to the lease payment schedule. To capture these updates, you need to re-create the payment schedule following the steps below.

> - When creating a new payment note the following:
>>  - The payment schedule will be based on the start/end dates of the selected lease. Payments cannot be added outside of this date range.
<br>
<br>
>>  - All upfront payments/prepayments or payments made upon termination are specified when the lease is created and automatically populated into the payment schedule on the first or last day of the lease. Any payments entered as occurring on the first or last day of the lease will be added to these amounts in the payment schedule.
<br>
<br>
>>  - To change the start/end date of the lease or the amounts of upfront/termination payments, go to <b> Step 2 </b> and edit the lease information, then return to this step and re-initialize the new payment schedule.

2. Run the cell below to initialize a new payment schedule.

In [229]:
lease_start = retreive('commencement_date')
lease_end = retreive('termination_date')
start_payments = retreive('upfront_payments')
end_payments = retreive('termination_payments')
    
pmts = payment_schedule(lease_start, lease_end, start_payments, end_payments)

3. After the payment schedule has been initialized, it can be viewed by running the cell below.

In [233]:
pmts.schedule

,payment_date,payment_amount
0,2020-04-01,0.0
1,2020-04-08,7200.0
2,2020-05-08,7200.0
3,2020-06-08,7200.0
4,2020-07-08,7200.0
5,2020-08-08,7200.0
6,2020-09-08,7200.0
7,2020-10-08,7200.0
8,2020-11-08,7200.0
9,2020-12-08,7200.0


4. The schedule can be edited by entering the appropriate parameters below and running the following two cells. To view the updated payment schedule, re-run the cell above. See detailed instructions for the parameters below. Once the payment schedule is prepared, continue to the next step.

In [231]:
payment_amount = 7200
payment_start_month = '2020-04'
day_of_month = 8
payment_repeat = 96
end_of_month = False

In [232]:
pmts.add_payments(payment_amount, payment_start_month, day_of_month, payment_repeat, end_of_month)

Payments have been added to the schedule.


None

> <b> Instructions: </b> To add payments to the schedule provide the following inputs:
> - <b>payment_amount:</b> This should be an integer or float value for the payment amount.
<br>
<br>
> - <b>payment_start_month:</b> This is the month the payment takes place, or the first month in a series of recurring payments. Enter it as a string in the 'YYYY-MM' format. 
<br>
<br>
> - <b>day_of_month:</b> This is an integer representing the day of the month that the payment or payments take place. If the payments occur on the last day of each month, then enter any number between 1 and 31 and set end_of_month to True.
<br>
<br>
> - <b>payment_repeat:</b> Use this if the payment will repeat on a monthly basis. If the payment will repeat on the same day of consecutive months (or on the last day of consecutive months), then set payment_repeat to the number of consecutive months that the payment will repeat for. If the payment is a one-time payment or made at an irregular interval, set payment_repeat to 1. The value should be an integer and must be greater than 0.
<br>
<br>
> - <b> end_of_month:</b> This is a boolean (True/False) value to indicate if the payment is made on the last day of the month. Note, this will also apply to repeated payments and will override any value in the day_of_month field.


The cell below will reset the payment schedule.

In [ ]:
reset = input("Are you sure you want to reset the payment schedule? (Y/N): ")

if reset == 'Y':
    pmts.reset_schedule()
    print('The schedule has been reset.')
else:
    print('The schedule was not reset.')

5. When the payment schedule is set up, run the cell below to save it. Note that if there is an existing payment schedule for the lease it will be overwritten. 

In [234]:
lease_id = retreive('lease_id')
company_id = retreive('company_id')

if len(pmt_df) > 0:
    overwrite = input("If you proceed, existing payment information will be overwritten. Would you like to continue? (Y/N):")
    if overwrite == 'Y':
        query = '''DELETE FROM payments WHERE lease_id = %(identifier)s;'''
        values = {'identifier':lease_id}
        try:
            write_query (db_connect, query, values)
            print('Previous payment information has been deleted.')
        except mysql.connector.Error as e:
            print('an error occurred with the database query and no changes were saved. See MySQL error below:')
            print(e)
else:
    overwrite = 'Y'

    
if overwrite == 'Y':
    
    query = '''
    INSERT INTO payments(
    company_id,
    lease_id,
    payment_date,
    payment_amount) VALUES (%s, %s, %s, %s);'''

    values = [(company_id, lease_id, str(date)[:10], amount) for date, amount in zip(pmts.schedule['payment_date'], pmts.schedule['payment_amount'])]

    try:
        write_query (db_connect, query, values, many = True)
        print('The payment schedule has been successfully updated.')
        del pmts
        del pmt_df
        del overwrite
    except mysql.connector.Error as e:
        print('an error occurred with the database query and no changes were saved. See MySQL error below:')
        print(e)
else:
    print("No changes were made to the lease's payment information")

The payment schedule has been successfully updated.


## Deleting Companies and Leases

Run the respective cells below to delete a company or a lease. Note that deleting a company will delete all leases associated with that company. Note that deleting a lease will delete all payment information associated with that lease. 

#### Delete a Company:
Run the cell below to delete the company (the company must be selected from <b>Step 1</b> above.)

In [235]:
company_name = companies.loc[companies['company_id']==company_id, 'company_name'].item()

delete = input(f"Are you sure you want to delete {company_name}? This is irreversible. (Y/N):")
    
if delete == 'Y':

    query = '''DELETE FROM companies WHERE company_id = %s;'''
    values = [company_id]

    try:
        write_query (db_connect, query, values)
        print(f"{company_name} has been deleted.")
        del delete

    except mysql.connector.Error as e:
        print('an error occurred with the database query and no changes were saved. See MySQL error below:')
        print(e)

else:
    print(f"{company_name} was not deleted.")

Are you sure you want to delete ABC Corporation? This is irreversible. (Y/N):Y
ABC Corporation has been deleted.


#### Delete a Lease:
Run the cell below to delete the lease (the lease must be selected from <b>Step 2</b> above.)

In [ ]:
delete = input(f"Are you sure you want to delete {lease}? This is irreversible. (Y/N):")
    
if delete == 'Y':

    query = '''DELETE FROM leases WHERE lease_name = %s;'''
    values = [lease]

    try:
        write_query (db_connect, query, values)
        print(f"{lease} has been deleted.")
        del delete

    except mysql.connector.Error as e:
        print('an error occurred with the database query and no changes were saved. See MySQL error below:')
        print(e)

else:
    print(f"{lease} was not deleted.")